In [105]:
import sys
!{sys.executable} -m pip install scapy==2.4.4
from scapy.all import *

class Reassembled_Packet(object):
    def __init__(self, packet):
        self.id = packet.id
        self.list_of_fragments = list()
        
    class fragment_sorted(object):
        def __call__(self, fn):
            def deco(self, *args):  
                fn(self, args)
                self.check_fragments(self.id)
            return deco

    @fragment_sorted()
    def add_fragment(self, packet):
        self.list_of_fragments.append(packet[0])
        
    def sort_fragments(self):
        self.list_of_fragments = sorted(self.list_of_fragments, key=lambda fragment: fragment.frag)
        self.reassemble()

    def check_fragments(self, unique_id):
        if len(self.list_of_fragments) > 1:
            if self.list_of_fragments[0].frag == 0 and (self.list_of_fragments[-1].flags & 'MF') == 0:
                print(f"Frame with ID: {unique_id} can reassemble")
                return True
        print(f"Frame with ID: {unique_id} cannot be reassemble yet")
        return False

def check_fragments_packet(filename):
    list_of_packet_id = {}
    frame = 0
    print(f"Network trace from file: {filename}")
    packet_list = rdpcap(filename)
    for packet in packet_list: # loop over the packet list
        frame += 1
        if packet.id in list_of_packet_id.keys(): 
            list_of_packet_id[packet.id].add_fragment(packet)
        else:
            new_rea_pkt = Reassembled_Packet(packet)
            new_rea_pkt.add_fragment(packet)
            list_of_packet_id[packet.id] = new_rea_pkt

check_fragments_packet('./pcap/fragmented_dns.pcap')         
check_fragments_packet('./pcap/Teardrop_Attack.pcap')

Network trace from file: ./pcap/fragmented_dns.pcap
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 can reassemble
Network trace from file: ./pcap/Teardrop_Attack.pcap
Frame with ID: 0 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
Frame with ID: 242 cannot be reassemble yet
